# Agenda-based search

First, the neighboring stations and their respective zones are obtained using an amended version of the code provided in the assignment:
The zone_dict also includes the line of the next station for a given station.

In [25]:
import pandas as pd
df = pd.read_csv('tubedata.csv', header=None)
df.head()

# Converts the zone from string to integer including outside london zone (Ex: Zone A)

def zone_to_int(zone):
         
    if zone == 'a': zone = 7
    elif zone == 'b': zone = 8
    elif zone == 'c': zone = 9
    elif zone == 'd': zone = 10
    else: 
        try:
            zone = int(zone)
        except:
            print('Unsuported Zone, Zone is None') #Just in case
            zone = None
    return zone

from collections import defaultdict
 
station_dict = defaultdict(list)
zone_dict = defaultdict(list)

# get data row by row
for index, row in df.iterrows():
  
  start_station = row[0]

  end_station = row[1]
  act_cost = int(row[3])

  line_name = row[2]  
    
  zone1 = row[4]
  zone2 = row[5]

  # station dictionary of child station tuples (child_name, cost from parent to the child)
  # {"Mile End": [("Stepney Green", 2), ("Wembley", 1)]}
  station_list = station_dict[start_station]
  station_list.append((end_station, act_cost,line_name))

  # the following two lines add the other direction of the tube "step"
  station_list = station_dict[end_station]
  station_list.append((start_station, act_cost,line_name))
  
  # we add the main zone
 # zone_dict[start_station].add(zone1)


  zone1 = zone_to_int(zone1)
  zone2 = zone_to_int(zone2)
  zone_dict[start_station].append(zone1)
  # we add the secondary zone
  if zone2 != 0:
    #zone_dict[start_station].add(zone2)
    zone_dict[start_station].append(zone2)
    # if the secondary zone is not 0 it's the main zone for the ending station
    #zone_dict[end_station].add(zone2)
    zone_dict[end_station].append(zone2)
  else:
    # otherwise the main zone for the ending station is the same as for the starting station
    #zone_dict[end_station].add(zone1)
    zone_dict[end_station].append(zone1)



In [26]:
## Testing 
station = 'Kensal Green'
print(station_dict[station])
print(zone_dict[station])
print("End_station is  : " , station )




[('Willesden Junction', 3, 'Bakerloo'), ("Queen's Park", 3, 'Bakerloo')]
[3, 2]
End_station is  :  Kensal Green


Implementation of DFS, BFS and UCS


#### Breadth First Search Implementation - **BFS**

In [27]:
def bfs_implementation( origin, destination, counter = 0):
  # Add current place to already_visited
  next_already_visited = [origin]
  # List of existent paths (for now only origin)
  total_paths = [[origin]]
  
  # Will perform exploration of all current paths
  while len(total_paths)!= 0:
    new_total_paths = []
    #Check all paths
    for path in total_paths:
      # Last element in path, where to go next?
      last_element_in_path = path[-1]
      counter += 1
        
      # goal check
      if destination in last_element_in_path: return path, counter
   
      #nodes_found = list( (neighb_stations(graph,last_element_in_path)))
      nodes_found = station_dict[last_element_in_path]
      if reverse: nodes_found.reverse()
      #iterates though neighboring stations:
      for node in nodes_found: 
                
        if node[0] not in next_already_visited:  # check if next node has been visited
          
          next_already_visited.append(node[0])
          # Adding possible path for further exploration
          new_total_paths = new_total_paths + [path + [node[0]]]     
    total_paths = new_total_paths
  # If solution does not exist
  return [],-1

def cost_caulator(path):
    path_size = len(path)
    total_cost = 0
    if path_size < 2:
        return 0
    for i in range(path_size - 1):
        nodes = station_dict[path[i]]
        for next_node, cost,line in nodes:
            if next_node == path[i+1]:
                total_cost+= cost
                continue
    return total_cost
  

In [28]:
print('BFS:')
reverse = False # 
bfs_path, count = bfs_implementation( 'Canada Water', 'Stratford',reverse)
print('bfs_path: ', bfs_path)
print('number of explorations = {}'.format(count))
print('Average time (cost) = ',cost_caulator(bfs_path))

BFS:
bfs_path:  ['Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Stratford']
number of explorations = 40
Average time (cost) =  15


#### Depth First Search Implementation - **DFS**

In [29]:
def DFS_implementation( start , goal,reverse):
    stack = [(start,[])]  # contains explored paths and nodes
    explored = []
    counts = 0

    while stack:
       
        current_node, current_path = stack.pop() # retrieves last element
        counts +=1
        
        if current_node == goal:
            current_path.append(current_node)
            cost = cost_caulator(current_path)
            return current_path, cost, counts
        if current_node not in explored:
            explored.append(current_node)
            nodes = station_dict[current_node]
            if reverse: nodes.reverse()
            for n in nodes:
                next_node = n[0]
                cost = n[1]
                stack.append((next_node,current_path +[(current_node)]))
    return None, counts

def cost_caulator(path):
    path_size = len(path)
    total_cost = 0
    if path_size < 2:
        return 0
    for i in range(path_size - 1):
        nodes = station_dict[path[i]]
        for next_node, cost,line in nodes:
            if next_node == path[i+1]:
                total_cost+= cost
                continue
    return total_cost
  

In [30]:
print('DFS:')
reverse = True
path,cost, counts = DFS_implementation( 'Canada Water', 'Stratford',reverse)
cost = cost_caulator(path)
print('Path =', path)
print('Average time (cost)  = ',cost)
print('Explored nodes = ',counts)

DFS:
Path = ['Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Aldgate East', 'Tower Hill', 'Aldgate', 'Liverpool Street', 'Bethnal Green', 'Mile End', 'Stratford']
Average time (cost)  =  28
Explored nodes =  616


In [31]:
print('DFS:')
reverse = False
path,cost, counts = DFS_implementation( 'Canada Water', 'Stratford',reverse)
cost = cost_caulator(path)
print('Path =', path)
print('Average time (cost)  = ',cost)
print('Explored nodes = ',counts)

DFS:
Path = ['Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Aldgate East', 'Tower Hill', 'Aldgate', 'Liverpool Street', 'Bethnal Green', 'Mile End', 'Stratford']
Average time (cost)  =  28
Explored nodes =  616


#### Uniform Cost Search - **UCS**

In [32]:
from queue import PriorityQueue
def UCS_implementation(origin, goal,extended,reverse = False):

    #Similar to BFS but has a priority queue
    paths_to_explore = PriorityQueue()
    visited_nodes = []

    #For UCS, the costs and line are added to the queue
    paths_to_explore.put(( 0,[origin],None)) #For UCS, the costs ans line are added to the queue
    # None is the beginning line for the first station
    nodes_explored = 0
    while (paths_to_explore.empty()==False):
        nodes_explored +=1
        actual_cost, path, current_line = paths_to_explore.get() 
        
        current_node = path[-1]
        
        
        visited_nodes.append(current_node)    
        
        if current_node == goal: 
            print('Nodes opened = ', nodes_explored )
           
            return path , actual_cost
       
        neighbours = station_dict[current_node]
        
        for nodes in neighbours:
            
            next_node = nodes[0]
            #cost of next node is nodes[1]
            new_line = nodes[2]
           
            cost_node = cost_calculator(actual_cost, nodes[1], current_line, new_line, extended)

            #cost_node = nodes[1] + actual_cost ## For non extended only
             
            new_path = path.copy()

            new_path.append(next_node)
           
           
            if next_node not in visited_nodes: paths_to_explore.put((cost_node, new_path,new_line))

def cost_calculator(initial_cost, cost_to_be_added, current_line, new_line, ucs_extended):
    if (ucs_extended == False or current_line == None or current_line == new_line ): return initial_cost+cost_to_be_added
    if current_line != new_line: return initial_cost+cost_to_be_added + 2


In [33]:
extended = False

UCS_path, cost = UCS_implementation( 'Canada Water', 'Stratford', extended,reverse)
print('path = ', UCS_path)
print('Average_time (cost)  = ',cost)

Nodes opened =  224
path =  ['Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green', 'Mile End', 'Stratford']
Average_time (cost)  =  14


In [34]:
extended = True

bfs_path, cost = UCS_implementation( 'Canada Water', 'Stratford', extended, reverse)
print(bfs_path)
print(cost)

Nodes opened =  69
['Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Stratford']
15


## Heuristic Based Search - Best First Seach **BSF**

In [35]:

from queue import PriorityQueue

def BFS_heuristic_implementation( origin, goal):

    paths_to_explore = PriorityQueue()
    visited_nodes = []
    paths_to_explore.put((0, 0,[origin]))  
    #First value, 0 is the heuristic cost
    #Second value, 0 is true cost- average time
    
    nodes_explored = 0
    #target_zone = get_zones(graph, goal )[1]
    target_zone = zone_dict[goal][1]

    print("Starting zone = ", zone_dict[origin][0] )
    print("Target zone = ", target_zone)
    
    while (paths_to_explore.empty()==False):
        
        h_cost,actual_cost, path = paths_to_explore.get()
        current_node = path[-1]
        visited_nodes.append(current_node)       
        
        if current_node == goal:  return path , actual_cost , nodes_explored 
            
        nodes_explored +=1
        neighbours = station_dict[current_node]
    
        for nodes in neighbours:
            
            next_node = nodes[0]
            cost_node =(actual_cost+ nodes[1])     
            new_path = path.copy()
            new_path.append(next_node)
            next_zone = zone_dict[next_node][0]
            h_cost = heuristic_calculator( target_zone, next_zone   ) 
            
            if next_node not in visited_nodes: paths_to_explore.put((h_cost,cost_node, new_path))
                

def heuristic_calculator( target_zone, next_zone   ):

    constant = 1
    h_cost = constant * (target_zone - next_zone)
    if cost < 0: return -1*h_cost
    else: return h_cost
    
    

In [36]:
print('Best First Search')
bfs_path, cost,nodes_explored = BFS_heuristic_implementation( 'Canada Water', 'Stratford')
print('Nodes opened = ', nodes_explored )
print('path = ',bfs_path)
print('Average_time (cost)  = ',cost)

Best First Search
Starting zone =  2
Target zone =  3
Nodes opened =  39
path =  ['Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Stratford']
Average_time (cost)  =  15


In [37]:
print('Best First Search')
bfs_path, cost,nodes_explored = BFS_heuristic_implementation( 'East Acton', 'Mile End')
print('Nodes opened = ', nodes_explored )
print('path = ',bfs_path)
print('Average_time (cost)  = ',cost)

Best First Search
Starting zone =  3
Target zone =  2


Nodes opened =  2836
path =  ['East Acton', 'White City', "Shepherd's Bush", 'Holland Park', 'Notting Hill Gate', 'Queensway', 'Lancaster Gate', 'Marble Arch', 'Bond Street', 'Green Park', 'Westminster', 'Waterloo', 'Southwark', 'London Bridge', 'Bermondsey', 'Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Bromley-by-Bow', 'Bow Road', 'Mile End']
Average_time (cost)  =  49


## Heuristic Based Search - A-Star (A*)

In [38]:

from queue import PriorityQueue

def a_star_implementation( origin, goal):

    paths_to_explore = PriorityQueue()
    visited_nodes = []
    paths_to_explore.put((0, 0,[origin],None))
    #First value, 0 is the heuristic cost
    #Second value, 0 is true cost- average time
    
    nodes_explored = 0
    #target_zone = get_zones(graph, goal )[1]
    target_zone = zone_dict[goal][1]

    print("Starting zone = ", zone_dict[origin][0] )
    print("Target zone = ", target_zone)
    
    while (paths_to_explore.empty()==False):

        #actual_cost, path, current_line = paths_to_explore.get()
        h_cost,actual_cost, path,current_line = paths_to_explore.get()
        current_node = path[-1]
        visited_nodes.append(current_node)       
        
        if current_node == goal:  return path , actual_cost , nodes_explored 
            
        nodes_explored +=1
        neighbours = station_dict[current_node]
    
        for nodes in neighbours:
            
            next_node = nodes[0]
            new_line = nodes[2]
            cost_node =(actual_cost+ nodes[1])     
            new_path = path.copy()
            new_path.append(next_node)
            next_zone = zone_dict[next_node][0]
            h_cost = heuristic_calculator( target_zone, next_zone   ) 
            g_cost = cost_calculator(actual_cost, nodes[1], current_line, new_line, extended)
            total_cost = h_cost + g_cost
            if next_node not in visited_nodes: paths_to_explore.put((total_cost,cost_node, new_path,new_line))
                

def heuristic_calculator( target_zone, next_zone   ):

    constant = 1
    h_cost = constant * (target_zone - next_zone)
    if cost < 0: return -1*h_cost
    else: return h_cost
    
def cost_calculator(initial_cost, cost_to_be_added, current_line, new_line, ucs_extended):
    if (ucs_extended == False or current_line == None or current_line == new_line ): return initial_cost+cost_to_be_added
    if current_line != new_line: return initial_cost+cost_to_be_added + 2

In [39]:
print('A* Search')
a_star_path, cost,nodes_explored = a_star_implementation( 'Canada Water', 'Stratford')
print('Nodes opened = ', nodes_explored )
print('path = ',a_star_path)
print('Average_time (cost)  = ',cost)

A* Search
Starting zone =  2
Target zone =  3
Nodes opened =  104
path =  ['Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Stratford']
Average_time (cost)  =  15


In [40]:
print('A* Search')
a_star_path, cost,nodes_explored = BFS_heuristic_implementation( 'East Acton', 'Mile End')
print('Nodes opened = ', nodes_explored )
print('path = ',a_star_path)
print('Average_time (cost)  = ',cost)

A* Search
Starting zone =  3
Target zone =  2
Nodes opened =  2836
path =  ['East Acton', 'White City', "Shepherd's Bush", 'Holland Park', 'Notting Hill Gate', 'Queensway', 'Lancaster Gate', 'Marble Arch', 'Bond Street', 'Green Park', 'Westminster', 'Waterloo', 'Southwark', 'London Bridge', 'Bermondsey', 'Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Bromley-by-Bow', 'Bow Road', 'Mile End']
Average_time (cost)  =  49


#  Genetic algorithm

In [41]:
import random, operator, time, itertools, math, copy
import numpy as np

In [42]:
import math
import hashlib
import string

def get_password(student_username, l=10):
    # Possible characters include upper-case English letters, numbers between 0 and 9 (inclusive), 
    # and the underscore symbol
    options = string.digits + string.ascii_uppercase  + "_"

    h = hashlib.sha256(("ECS759P-AI"+student_username).encode("utf-8"))
    d = h.digest()
    s = ""
    for n in d:
      s += options[n%len(options)]

    return s[0:l]

# TO DO: replace *** with your EECS username and uncomment the code
student_password = get_password('ec23832')
print(student_password)

# Distance function
def distance_function(string_one, string_two):
    score = 0
    for i, j in zip(string_one, string_two):
        # Square of the absolute difference between two Unicode codes
        score += math.sqrt(abs(ord(i) - ord(j)))
    return score


# Upper bound of the distance value
MAX_VALUE = distance_function('0000000000', '__________')

# Compute normalised fitness for a list of candidate passwords 
def get_normalised_fitness(list_of_phrases, student_password):
    ordered_dict = dict()
    phrase_to_find = student_password
    for phrase in list_of_phrases:
        # Return 1 when a candidate matches the true password (string distance between them is zero)
        ordered_dict[phrase] = 1 - distance_function(phrase, phrase_to_find) / MAX_VALUE
    return ordered_dict

# Example of how to get fitness values for a list of candidates
get_normalised_fitness(['2Q4HHHHOTJ', '2HHZQYUOTJ'], student_password)

V3Q520P12R


{'2Q4HHHHOTJ': 0.3031195549479525, '2HHZQYUOTJ': 0.2997182297537069}

The following cell is about random password and initial population generation

In [43]:
password_len = 10

pool = []

# The function rand_password() creates a random password of the samesize of the original password
def rand_password():
    pool = "01234567890ABCDEFGHIJKLMNOPQRSTUVWXYZ_"
    password_len = 10
    rand_password = ''
    for i in range(password_len):
        index = random.randrange(0, len(pool))
    
        rand_password += pool[index] 
        #rand_password.append(pool[index])
    return rand_password

# This functions creates a list of random passwords generated of a given population size
def create_init_population(pop_size):
    '''
    Method to create initial population
    '''
    population = []

    for i in range(0, pop_size):
        population.append(rand_password())
        
    return population


Mutation and crossover functions declaration

In [44]:
def mutation(pool_to_mutate, pMuta = 0.5):
 
  # Mutate provided individuals with pMuta probability
 
  mutated_pool = [] #list to save the mutated and non mutated 'password'
  for index, indiv in enumerate(pool_to_mutate):
    indiv = list(indiv) # convert indiv from str to list  'str' object does not support item assignment
    for pos in range(len(indiv)):  #pos is a char in the password
      if np.random.rand() <= pMuta:
        # pick a random position to swap with
        pos2 = random.sample(range(len(indiv)), 1)[0]
        # swap the positions
        temp_char = indiv[pos]
        indiv[pos] = indiv[pos2]
        indiv[pos2] = temp_char
    indiv = ''.join(indiv) # convert indiv back to str
    mutated_pool.append(indiv)
  return mutated_pool


def crossing_over(pool_to_cross, pCO = 0.1,reproduction = 0):
 
  # Applies crossing over on the selected individuals with a probability pCO
   
  for index, indiv in enumerate(pool_to_cross):
    indiv_ls = list(indiv)  # convert indiv from str to list  'str' object does not support item assignment
 
    if np.random.rand() <= pCO:
      # We get the list of all the other selected individuals
      others = pool_to_cross[:]

      others.remove(indiv)
      # We pick randomly one of them
      other = others[np.random.choice(len(others))]
      # We get its index in order to modify it directly
      otherIdx = pool_to_cross.index(other)
      # Randomly choose a starting point and
      # then we swap the genome starting from that position
      # between the two individuals
      startingIdx = np.random.choice(len(indiv))
      tempChange = indiv[startingIdx:]

      # we check if the permutation property is preserved
      # (i.e., cities will not be repeated after cross-over)
      if not any(character in indiv[:startingIdx] for character in other[startingIdx:]):
        indiv = list(indiv)
        other = list(other)
        indiv[startingIdx:] = other[startingIdx:]
        other[startingIdx:] = tempChange
        # Modification of the new one
        indiv = ''.join(indiv)
        other = ''.join(other)  
        pool_to_cross[otherIdx] = other
        
        pool_to_cross[index] = indiv
        reproduction+=1
    
  return pool_to_cross, reproduction

In [45]:
# this function sort the fitness and returns the sorted_population as a list
# It also returns the best population and its repective fitness
def sort_individuals(fitness):
  fitness = sorted(fitness.items(), key=lambda x:1-x[1]) # sorted it in a list
  best_fitness = fitness[0]
  sorted_population = []
  for i in fitness: sorted_population.append(i[0])
    
  return sorted_population , best_fitness

## The Genetic Algorithm

In [46]:
# The genetic algorithm


pool = "01234567890ABCDEFGHIJKLMNOPQRSTUVWXYZ_"
# Population size
pop_size = 300
half = int(pop_size // 2)
# Mutation rate (per gene)

reproduction = 0

pMuta = 0.3
# Crossover rate
pCO = 0.8
# Maximum Number of generations
n_gen = 400

# initializing the population
population = create_init_population(pop_size)
#getting the fitness
fitness = get_normalised_fitness(population, student_password)
# arranging the population with highest fitness as starting position
population, best_fitness = sort_individuals(fitness)


print("Best initial population is ", best_fitness[0], " and has a fitness of ", best_fitness[1] )

for i in range(n_gen):

  # the mating pool is the top halved of the population
  # the bottom halved (least fitness) are eliminated; survival of fitness
  mating_pool = copy.deepcopy(population)[:half]
    
  # perform cross over and mutation over the best parents
  mating_pool,reproduction = crossing_over(mating_pool, pCO,reproduction)
  mating_pool = mutation(mating_pool, pMuta)
    
  # combine the best of parents and offsprings to form a new population
  population = copy.deepcopy(population)[:half] + mating_pool
    
  # Sort population according to fitness 
  fitness = get_normalised_fitness(population, student_password)
  population, best_fitness = sort_individuals(fitness)
  print(best_fitness)
  if best_fitness[1] == float(1.0):
      break
print('Epochs = ',i+1)
print('Reproduction = ', reproduction)
print("Best result is ", best_fitness[0], " and has a fitness of ", best_fitness[1] )

Best initial population is  UFWP19N05P  and has a fitness of  0.6708553689384302


('UFWP19N05P', 0.6708553689384302)
('U3K3613000', 0.6864772976128084)
('M2Q7Q7A04P', 0.6888825217948757)
('W6QM10707G', 0.7055901407312773)
('W6QM10707G', 0.7055901407312773)
('Q7Q07DP12E', 0.7551526128229828)
('U2MA10Q75V', 0.7717848538551728)
('U2MA10Q75V', 0.7717848538551728)
('V3R139_07P', 0.7735699099372332)
('V3R139_07P', 0.7735699099372332)
('B3Q631K07X', 0.7754589087396289)
('U2M4A0P77Q', 0.7876418531114632)
('S1Q473O00Q', 0.8172516346338272)
('S1Q473O00Q', 0.8172516346338272)
('S1Q473O00Q', 0.8172516346338272)
('X2R770Q43P', 0.8218877673406468)
('X2R770Q43P', 0.8218877673406468)
('X2R770Q43P', 0.8218877673406468)
('X2R770Q43P', 0.8218877673406468)
('V4N200L40Q', 0.8246034806545037)
('B3K630Q12S', 0.8406917160449732)
('B3K630Q12S', 0.8406917160449732)
('B3K630Q12S', 0.8406917160449732)
('V2Q014P23R', 0.8798646014485911)
('V2Q014P23R', 0.8798646014485911)
('V2Q014P23R', 0.8798646014485911)
('V3Q124P20R', 0.9064390793258769)
('V3Q124P20R', 0.9064390793258769)
('V3Q124P20R', 0.906

#### Mean and STD for reproduction and epochs
First the Genetic algoritm is written as a function

In [47]:
def genetic_algo(pop_size = 300, pMuta = 0.3, pCO = 0.8 ,n_gen = 400):

    pool = "01234567890ABCDEFGHIJKLMNOPQRSTUVWXYZ_"
    
    population = create_init_population(pop_size)
    fitness = get_normalised_fitness(population, student_password)
    population, best_fitness = sort_individuals(fitness)
    reproduction = 0
    for i in range(n_gen):
      
      mating_pool = copy.deepcopy(population)[:half]
      mating_pool,reproduction = crossing_over(mating_pool, pCO,reproduction)
      mating_pool = mutation(mating_pool, pMuta)
    
      population = copy.deepcopy(population)[:half] + mating_pool
        
      fitness = get_normalised_fitness(population, student_password)
      population, best_fitness = sort_individuals(fitness)
     
      if best_fitness[1] == float(1.0):
          break
    return reproduction , i

def mean(list_, samples):
    mean = 0
    for i in range(samples):
        mean += list_[i]
    return mean / samples
def STD(list_,mean,samples):
    sum = 0
    for i in range(samples):
        sum += (list_[i] - mean)**2
    return (sum/samples)**0.5

In [48]:
samples = 10
reproductions = []
l_epochs = []
for s in range(samples):
    reproductions.append(genetic_algo()[0])
    l_epochs.append(genetic_algo()[1])

mean_reproduction = mean(reproductions, samples)
mean_epochs= mean( l_epochs, samples)
std_reproduction = STD(reproductions,mean_reproduction,samples)
std_epochs = STD(l_epochs,mean_epochs,samples)
print("Number of samples are: ",samples)
print("The mean of reproduction is: ", mean_reproduction)
print("The standard deviation of reproduction is: ", std_reproduction)
print("The mean of epochs is: ", mean_epochs)
print("The standard deviation of epochs is: ", std_epochs)

Number of samples are:  10
The mean of reproduction is:  12413.8
The standard deviation of reproduction is:  5059.545963819283
The mean of epochs is:  208.5
The standard deviation of epochs is:  70.75485849042452
